
# `Lib-INVENT`: Reinforcement Learning - ROCS + reaction filter
The purpose of this notebook is to illustrate the assembly of a configuration input file containing a ROCS input.

ROCS is a licensed virtual screening software based on similarity between input compounds and a specified reference (or target) molecule. For more information on ROCS, please refer to the OpenEye website: https://www.eyesopen.com/rocs

*Note that in order to use ROCS, an OpenEye license is needed.*

In [29]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
project_directory = "</path/to/project/directory>"
output_directory = "</path/to/output/directory>"

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_directory)
except FileExistsError:
    pass

## Setting up the configuration
The configuration is set up analogously to the previous tutorials. The difference arrises in the scoring function composition where a QSAR predictive property is not replaced with the ROCS model.

The previously discussed selective Amide-coupling/Buchwald reaction filters are imposed.

### 1. Run Type Block

In [30]:
# initialize the dictionary
configuration = {
    "run_type": "reinforcement_learning"
}

### 2. Logging Block

In [31]:
# Add logging
configuration.update({
    "logging": {
        "sender": " ",        
        "recipient": "local",                
        "logging_path": os.path.join(output_directory, "run.log"), 
        "job_name": "RL Demo ROCS",                
        "job_id": " "   
        }
    })

### 3. Parameters Block

In [32]:
#start assembling parameters block
configuration.update({
    "parameters": {
        "actor": os.path.join(project_directory, "trained_models/reaction_based.model"),
        "critic": os.path.join(project_directory, "trained_models/reaction_based.model"),
        "scaffolds": ["[*:0]N1CCN(CC1)CCCCN[*:1]"],
        "n_steps": 100,
        "learning_rate": 0.0001,
        "batch_size": 128,
        "randomize_scaffolds": False # important since a RF is to be imposed.
    }
})

In [33]:
#configure learning strategy
learning_strategy = {
        "name": "DAP",
        "parameters":  {
        "sigma": 120
    }
}

configuration["parameters"]["learning_strategy"] = learning_strategy

#### Scoring strategy

In [34]:
# configure scoring strategy
scoring_strategy = {
    "name": "standard" 
}

In [35]:
# configure diversity filter
diversity_filter =  {
    "name": "NoFilterWithPenalty",     # To use a DF. The alternative option is "NoFilter"
}

scoring_strategy["diversity_filter"] = diversity_filter

In [36]:
# configure reaction filter
reaction_filter =  {
    "type":"selective",        
    "reactions":{"1": ["[#6;!$(C(C=*)(C=*));!$([#6]~[O,N,S]);$([#6]~[#6]):1][C:2](=[O:3])[N;D2;$(N(C=[O,S]));!$(N~[O,P,S,N]):4][#6;!$(C=*);!$([#6](~[O,N,S])N);$([#6]~[#6]):5]>>[#6:1][C:2](=[O:3])[*].[*][N:4][#6:5]"],
                 "0": ["[c;$(c1:[c,n]:[c,n]:[c,n]:[c,n]:[c,n]:1):1]-!@[N;$(NC)&!$(N=*)&!$([N-])&!$(N#*)&!$([ND1])&!$(N[O])&!$(N[C,S]=[S,O,N]),H2&$(Nc1:[c,n]:[c,n]:[c,n]:[c,n]:[c,n]:1):2]>>[*][c;$(c1:[c,n]:[c,n]:[c,n]:[c,n]:[c,n]:1):1].[*][N:2]"]}
}

scoring_strategy["reaction_filter"] = reaction_filter

#####  Define the scoring function
This is the important point of difference to the previous tutorials. A ROCS similarity measure is imposed to guide the agent to propose compounds structurally resembling a known haloperidol ligand.

The inputs used for the publication is provided in the tutorial/models/ROCS/ folder.



In [37]:
scoring_function = {
    "name": "custom_sum",              
    "parallel": False,                     
    "parameters": [
    {
        "component_type": "parallel_rocs_similarity",
        "model_path": None,
        "name": "RefTversky ROCS sim",
        "smiles": [],
        "specific_parameters": {
            "color_weight": 0.5,
            "custom_cff": "<project_directory>/tutorial/models/ROCS/implicit_MD_mod_generic_hydrophobe.cff",
            "enumerate_stereo": True,
            "input_type": "shape_query",
            "max_num_cpus": 8,
            "rocs_input": "<project_directory>/tutorial/models/ROCS/haloperidol_3_feats.sq",
            "shape_weight": 0.5,
            "similarity_measure": "RefTversky",
            "transformation_type": "no_transformation"
        },
        "weight": 1
    },
    {
        "component_type": "custom_alerts",
        "name": "Custom alerts",              
        "weight": 1,                          
        "model_path": None,                    
        "smiles": [                           
            "[*;r8]",
            "[*;r9]",
            "[*;r10]",
            "[*;r11]",
            "[*;r12]",
            "[*;r13]",
            "[*;r14]",
            "[*;r15]",
            "[*;r16]",
            "[*;r17]",
            "[#8][#8]",
            "[#6;+]",
            "[#16][#16]",
            "[#7;!n][S;!$(S(=O)=O)]",
            "[#7;!n][#7;!n]",
            "C#C",
            "C(=[O,S])[O,S]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#7;!n]",
            "[#7;!n][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#16;!s]",
            "[#8;!o][C;!$(C(=[O,N])[N,O])][#8;!o]",
            "[#16;!s][C;!$(C(=[O,N])[N,O])][#16;!s]"
        ],
        "specific_parameters": None           
    }]
}

scoring_strategy["scoring_function"] = scoring_function

In [1]:
# Update the paramters block with the scoring strategy
configuration["parameters"]["scoring_strategy"] = scoring_strategy

NameError: name 'scoring_strategy' is not defined

## Write out the configuration

In [39]:
configuration_JSON_path = os.path.join(output_directory, "input.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

# Run
Execute in jupyter notebook

In [40]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!python {project_directory}/input.py {configuration_JSON_path}

In [41]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

Execute in command line
```
$ conda activate lib-invent
$ python <project_directory>/input.py <configuration_JSON_path>
```
